In [138]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"data\maleChemicalAdjMatrix.csv")
df.head(5)

,Unnamed: 0,I1L,I1R,I2L,I2R,I3,I4,I5,I6,M1,...,vsrR,gonad,proctodeum,R1stR,R2stR,R3stL,R5stL,R7stL,R7stR,R9stL
0,I1L,NaN,NaN,10.0,NaN,3.0,NaN,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I1R,NaN,NaN,NaN,6.0,1.0,NaN,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I2L,2.0,NaN,NaN,3.0,NaN,13.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,I2R,NaN,1.0,3.0,NaN,NaN,20.0,2.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,I3,1.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
df.shape

(382, 576)

In [140]:
matrix = df.set_index(df.columns[0])
edges = matrix.stack().dropna()
edge_list = edges.index.to_list()

In [141]:
matrix.shape
len(edge_list)

5306

get rid of duplicate edges

In [142]:
un_edges = []
for n1, n2 in edge_list:
    if ((n2, n1) not in un_edges and (n1, n2) not in un_edges):
        un_edges.append((n1, n2))

Create a unique index for each node.

In [143]:
unique_nodes = {}
int_edges = []
i = 0

for node1, node2 in un_edges:
    if node1 not in unique_nodes:
        unique_nodes[node1] = i
        i += 1
    if node2 not in unique_nodes:
        unique_nodes[node2] = i
        i += 1

    int_edges.append((unique_nodes[node1], unique_nodes[node2]))

print(unique_nodes)

{'I1L': 0, 'I2L': 1, 'I3': 2, 'I5': 3, 'I6': 4, 'M2L': 5, 'M3L': 6, 'M3R': 7, 'MCL': 8, 'MCR': 9, 'MI': 10, 'NSML': 11, 'pm1': 12, 'pm3VL': 13, 'pm4D': 14, 'pm4VL': 15, 'I1R': 16, 'I2R': 17, 'M1': 18, 'M2R': 19, 'NSMR': 20, 'pm3D': 21, 'pm3VR': 22, 'pm4VR': 23, 'pm5D': 24, 'pm5VR': 25, 'I4': 26, 'M4': 27, 'pm5VL': 28, 'e3VL': 29, 'e3VR': 30, 'M5': 31, 'pm6VL': 32, 'g1AL': 33, 'g1AR': 34, 'g1p': 35, 'bm': 36, 'pm2D': 37, 'pm2VL': 38, 'e3D': 39, 'g2L': 40, 'pm6D': 41, 'pm6VR': 42, 'pm7D': 43, 'pm7VR': 44, 'mc3V': 45, 'g2R': 46, 'mc2DR': 47, 'mc2V': 48, 'mc2DL': 49, 'ASIL': 50, 'ASGL': 51, 'AFDL': 52, 'AWCL': 53, 'AIML': 54, 'AIAL': 55, 'AIBL': 56, 'CEPshDL': 57, 'CEPshVL': 58, 'ASIR': 59, 'ASGR': 60, 'AFDR': 61, 'AWCR': 62, 'ASKR': 63, 'AIAR': 64, 'ASJL': 65, 'ASKL': 66, 'OLQDL': 67, 'PVQL': 68, 'URADL': 69, 'ASJR': 70, 'PVQR': 71, 'RIFR': 72, 'RMGR': 73, 'AVBL': 74, 'AWAL': 75, 'AWBL': 76, 'ASEL': 77, 'ADFL': 78, 'AIYL': 79, 'AUAL': 80, 'AIZL': 81, 'RIBL': 82, 'AWAR': 83, 'AWBR': 84, 'A

In [144]:
un_set = set(unique_nodes)
un_cols = set(df.columns[1:])

print(un_cols.difference(un_set))

{'dglL3', 'dglR4', 'dsrR', 'BAGR', 'int', 'GLRR', 'GLRDR', 'dsrL', 'vsrL', 'ADAR', 'GLRDL', 'pilR', 'pilL', 'vsrR', 'GLRL', 'CA01'}


all these nodes actually have no edges. So we aren't going to worry about them

In [145]:
df['dglL3'][~df['dglL3'].isna()]

Series([], Name: dglL3, dtype: float64)

New edge list with integer-valued nodes.

In [146]:
col_df = pd.read_excel(r"data\maleChemicalFunctionGroupColoringByColumn.xlsx", header=None)
print(col_df.head())
print(col_df.shape)

     0  1
0  I1L  0
1  I1R  0
2  I2L  0
3  I2R  0
4   I3  0
(575, 2)


get the functional groupings from the data.

In [147]:
# zip(keys, values)
# make sure that nodes with no edges aren't actually in the data set
color_assignment = dict(zip(col_df.iloc[:, 0], col_df.iloc[:, 1]))
node_diff = list(un_cols.difference(un_set)) 

for node in node_diff:
    color_assignment.pop(node)

final_dict = {}

for key, value in color_assignment.items():
    try:
        int_key = unique_nodes[key]
        final_dict[int_key] = str(value)
    except Exception as e:
        print(e)

final_dict

{0: '0',
 16: '0',
 1: '0',
 17: '0',
 2: '0',
 26: '0',
 3: '0',
 4: '0',
 18: '0',
 5: '0',
 19: '0',
 6: '0',
 7: '0',
 27: '0',
 31: '0',
 8: '0',
 9: '0',
 10: '0',
 11: '0',
 20: '0',
 12: '0',
 37: '0',
 38: '0',
 21: '0',
 13: '0',
 22: '0',
 14: '0',
 15: '0',
 23: '0',
 24: '0',
 28: '0',
 25: '0',
 41: '0',
 32: '0',
 42: '0',
 43: '0',
 44: '0',
 49: '0',
 47: '0',
 48: '0',
 45: '0',
 39: '0',
 29: '0',
 30: '0',
 35: '0',
 33: '0',
 34: '0',
 40: '0',
 46: '0',
 36: '0',
 50: '1',
 59: '1',
 65: '1',
 70: '1',
 75: '1',
 83: '1',
 51: '1',
 60: '1',
 76: '1',
 84: '1',
 77: '1',
 96: '1',
 78: '1',
 85: '1',
 52: '1',
 61: '1',
 53: '1',
 62: '1',
 66: '1',
 63: '1',
 104: '1',
 106: '1',
 107: '1',
 86: '1',
 118: '1',
 121: '1',
 123: '1',
 128: '1',
 134: '1',
 139: '1',
 144: '1',
 146: '1',
 147: '1',
 156: '1',
 159: '1',
 179: '1',
 148: '1',
 185: '1',
 187: '1',
 192: '1',
 195: '1',
 149: '1',
 150: '1',
 151: '1',
 205: '1',
 206: '1',
 157: '1',
 212: '1',
 18

compile color assignment to list, indexed by the node integer value

In [148]:
final_colors = list(dict(sorted(final_dict.items())).values())


In [156]:
with open(r"data\celegans_edges.txt", "w") as file:
    file.write(str(int_edges))

with open(r"data\celegans_colors.txt", "w") as file:
    file.write(str(final_colors))

In [150]:
for key, value in unique_nodes.items():
    if value == 283:
        print(key, value)
        break

IL1DR 283
